In [1]:
import qutip as qt
import numpy as np
print(qt.__version__)


5.0.4


## First we define state and density matrix

In [2]:


def create_w_state(n):
    """Creates an n-qubit W state."""
    basis_states = [qt.basis([2]*n, [1 if i == j else 0 for i in range(n)]) for j in range(n)]
    w_state = sum(basis_states) / np.sqrt(n)
    return w_state

def create_entangled_pairs_state():
    """Creates the 3-qubit entangled pairs state: (|010⟩ + |101⟩) / sqrt(2)."""
    basis_010 = qt.basis([2,2,2], [0,1,0])  # |010⟩
    basis_101 = qt.basis([2,2,2], [1,0,1])  # |101⟩
    entangled_state = (basis_010 + basis_101) / np.sqrt(2)
    return entangled_state

def create_GHZ_state():
    basis_111 = qt.basis([2,2,2],[1,1,1])
    basis_000 = qt.basis([2,2,2],[0,0,0])
    GHZ_state = (basis_111 + basis_000)/np.sqrt(2)
    return(GHZ_state)


def compute_density_matrix(state):
    """Computes the density matrix of a pure state."""
    return state * state.dag()

num_qubits = 3

# Create W state and its density matrix
w_state = create_w_state(num_qubits)
p_W = compute_density_matrix(w_state)

entangled_state = create_entangled_pairs_state()
p_E = compute_density_matrix(entangled_state)

GHZ_State = create_GHZ_state()
p_G = compute_density_matrix(GHZ_State)

# Compute the mixed state: ρ_mixed = 0.5 * p_W + 0.5 * p_E
p_full = (1/3)*( p_W +  p_E + p_G)

# Print results
print("Mixed State Density Matrix (0.5 * W State + 0.5 * Entangled Pairs State):")
np.set_printoptions(linewidth=200, precision=3, suppress=True)
print(np.real(p_full.full()))



Mixed State Density Matrix (0.5 * W State + 0.5 * Entangled Pairs State):
[[0.167 0.    0.    0.    0.    0.    0.    0.167]
 [0.    0.111 0.111 0.    0.111 0.    0.    0.   ]
 [0.    0.111 0.278 0.    0.111 0.167 0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.111 0.111 0.    0.111 0.    0.    0.   ]
 [0.    0.    0.167 0.    0.    0.167 0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.167 0.    0.    0.    0.    0.    0.    0.167]]


## Conditional Mutual Information bound check

In [3]:
def compute_reduced_density_matrices(p_W):
    p_A = p_full.ptrace([0])  # Tracing out B and C
    p_B = p_full.ptrace([1])  # Tracing out A and C
    p_C = p_full.ptrace([2])  # Tracing out A and B
    p_AB = p_full.ptrace([0, 1])  # Tracing out C
    p_AC = p_full.ptrace([0, 2])  # Tracing out B
    p_BC = p_full.ptrace([1, 2])  # Tracing out A
    return p_A, p_B, p_C, p_AB, p_AC, p_BC


p_A, p_B, p_C, p_AB, p_AC, p_BC = compute_reduced_density_matrices(p_W)

def compute_entropies(p_A, p_B, p_C, p_AB, p_AC, p_BC, p_W):
    S_A = qt.entropy_vn(p_A)
    S_B = qt.entropy_vn(p_B)
    S_C = qt.entropy_vn(p_C)
    S_AB = qt.entropy_vn(p_AB)
    S_AC = qt.entropy_vn(p_AC)
    S_BC = qt.entropy_vn(p_BC)
    S_W = qt.entropy_vn(p_W)  # Total system entropy
    return S_A, S_B, S_C, S_AB, S_AC, S_BC, S_W

S_A, S_B, S_C, S_AB, S_AC, S_BC, S_full = compute_entropies(p_A, p_B, p_C, p_AB, p_AC, p_BC, p_full)


In [4]:
# Nicely formatted printout
entropy_labels = ["S(ρ_A)", "S(ρ_B)", "S(ρ_C)", "S(ρ_AB)", "S(ρ_AC)", "S(ρ_BC)", "S(ρ_full)"]
entropy_values = [S_A, S_B, S_C, S_AB, S_AC, S_BC, S_full]

# Print header
print("=" * 40)
print(f"{'Subsystem':<10} | {'Entropy':>12}")
print("=" * 40)

# Print entropy values
for label, value in zip(entropy_labels, entropy_values):
    print(f"{label:<10} | {value:12.6f}")

print("=" * 40)


Subsystem  |      Entropy
S(ρ_A)     |     0.686962
S(ρ_B)     |     0.686962
S(ρ_C)     |     0.686962
S(ρ_AB)    |     1.320359
S(ρ_AC)    |     1.060857
S(ρ_BC)    |     1.320359
S(ρ_full)  |     1.041400


## Compute the Conditional Mutual Information

In [5]:
I_ABcC = S_AC + S_BC - S_full - S_C
print("Conditional mutual information I(A:B|C) =",I_ABcC)

Conditional mutual information I(A:B|C) = 0.6528537287359995


## Compute 3 possible bounds using Z operator

In [6]:
Z = qt.sigmaz()
Z_2 = qt.tensor(Z, Z)
Z_full = qt.tensor(Z, Z, Z)



avg_Z_A = (p_A @ Z).tr()
avg_Z_B = (p_B @ Z).tr()
avg_Z_C = (p_C @ Z).tr()
avg_Z_AC = (p_AC @ Z_2).tr()
avg_Z_BC = (p_BC @ Z_2).tr()
avg_Z_AB = (p_AB @ Z_2).tr()
avg_Z_full = (p_W @ Z_full).tr()

Z_norm = 1



In [7]:
# Define labels and values
z_labels = [
    "⟨Z_A⟩", "⟨Z_B⟩", "⟨Z_C⟩",
    "⟨Z_AC⟩", "⟨Z_BC⟩", "⟨Z_AB⟩",
    "⟨Z_W⟩ (Full)"
]
z_values = [
    avg_Z_A, avg_Z_B, avg_Z_C,
    avg_Z_AC, avg_Z_BC, avg_Z_AB,
    avg_Z_full
]

# Print header
print("=" * 40)
print(f"{'Observable':<10} | {'Expectation Value':>20}")
print("=" * 40)

# Print expectation values
for label, value in zip(z_labels, z_values):
    print(f"{label:<10} | {value:20.6f}")

print("=" * 40)


Observable |    Expectation Value
⟨Z_A⟩      |             0.111111
⟨Z_B⟩      |             0.111111
⟨Z_C⟩      |             0.111111
⟨Z_AC⟩     |             0.555556
⟨Z_BC⟩     |            -0.111111
⟨Z_AB⟩     |            -0.111111
⟨Z_W⟩ (Full) |            -1.000000


In [11]:
f_1 = 0.5*(((avg_Z_full-(avg_Z_A*avg_Z_BC))**2 + (avg_Z_full-(avg_Z_B*avg_Z_AC))**2)/(2*3*Z_norm**2)-4)
f_2 = (avg_Z_full-avg_Z_A*avg_Z_B*avg_Z_C)**2/(2*3*Z_norm**2)-2
f_3 = (avg_Z_AB-avg_Z_A*avg_Z_B)**2/(2*2*Z_norm**2) 
f_4 = (avg_Z_full*avg_Z_C - avg_Z_AC*avg_Z_BC)**2/(2*4*Z_norm**2)

print(" f_1 = ", np.real(f_1),"\n f_2 = ", np.real(f_2),"\n f_3 =",np.real(f_3),"\n f_4 = ", np.real(f_4))

 f_1 =  -1.8247726464461718 
 f_2 =  -1.8328757723497684 
 f_3 = 0.003810394756896815 
 f_4 =  0.00030483158055174506


In [10]:
W = S_BC+S_AC-S_C-S_full
X = S_A+S_AC-S_C-S_full
Y = S_A + S_B-S_C-S_full
Z = S_A+S_B-S_AB-S_full
print("\nW =",W,"\nX =",X,"\nY =",Y,"\nZ =",Z)


W = 0.6528537287359995 
X = 0.01945656619084457 
Y = -0.3544388043698534 
Z = -0.9878359669150083


In [24]:
M = qt.sigmaz()
I = qt.qeye(2) 
M_2 = (qt.tensor(M, I) + qt.tensor(I, M)) / 2
M_full = (qt.tensor(M, I, I) + qt.tensor(I, M, I) + qt.tensor(I, I, M)) / 3


avg_M_A = (p_A @ M).tr()
avg_M_B = (p_B @ M).tr()
avg_M_C = (p_C @ M).tr()
avg_M_AC = (p_AC @ M_2).tr()
avg_M_BC = (p_BC @ M_2).tr()
avg_M_AB = (p_AB @ M_2).tr()
avg_M_full = (p_W @ M_full).tr()

np.set_printoptions(linewidth=200, precision=3, suppress=True)
print("\nDensity MatriM in MatriM Form:")
print(np.real(p_AB.full()))

np.set_printoptions(linewidth=200, precision=3, suppress=True)
print("\nDensity MatriM in MatriM Form:")
print(np.real(M_2.full()))

M_norm = 1
# Define labels and values
M_labels = [
    "⟨M_A⟩", "⟨M_B⟩", "⟨M_C⟩",
    "⟨M_AC⟩", "⟨M_BC⟩", "⟨M_AB⟩",
    "⟨M_W⟩ (Full)"
]
M_values = [
    avg_M_A, avg_M_B, avg_M_C,
    avg_M_AC, avg_M_BC, avg_M_AB,
    avg_M_full
]

# Print header
print("=" * 40)
print(f"{'Observable':<10} | {'EMpectation Value':>20}")
print("=" * 40)

# Print eMpectation values
for label, value in zip(M_labels, M_values):
    print(f"{label:<10} | {value:20.6f}")

print("=" * 40)


Density MatriM in MatriM Form:
[[0.278 0.    0.    0.   ]
 [0.    0.278 0.111 0.   ]
 [0.    0.111 0.278 0.   ]
 [0.    0.    0.    0.167]]

Density MatriM in MatriM Form:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
Observable |    EMpectation Value
⟨M_A⟩      |             0.111111
⟨M_B⟩      |             0.111111
⟨M_C⟩      |             0.111111
⟨M_AC⟩     |             0.111111
⟨M_BC⟩     |             0.111111
⟨M_AB⟩     |             0.111111
⟨M_W⟩ (Full) |             0.333333


 f_1 =  -0.9828278209622516 
 f_2 =  -1.9816335836590202 
 f_3 = -1.0389617283227628 
 f_4 =  7.620789513793616e-05
